In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import numpy as np
import ray
from ray.rllib import agents
from tqdm.notebook import tqdm
import random
from ray.rllib.policy.policy import Policy
from gym.spaces import Discrete, Box
from ray.rllib.agents.ppo import PPOTrainer
from functools import partial
from ray.tune.logger import pretty_print
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy
from ray.rllib.models import ModelCatalog


import ray
from ray import tune
from ray.tune import track

import math
import gym

from gym_compete_to_rllib import created_envs, env_name, create_env, env_name_rllib

import os
os.environ['DISPLAY'] = ':0'

In [2]:
def ray_init():
    ray.shutdown()
    return ray.init(ignore_reinit_error=True,
                    temp_dir='/scratch/sergei/tmp') # Skip or set to ignore if already called

ray_init()


2020-07-17 19:50:37,780	INFO resource_spec.py:223 -- Starting Ray with 152.1 GiB memory available for workers and up to 69.19 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-07-17 19:50:38,556	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '128.32.175.10',
 'raylet_ip_address': '128.32.175.10',
 'redis_address': '128.32.175.10:19203',
 'object_store_address': '/scratch/sergei/tmp/session_2020-07-17_19-50-37_777157_55814/sockets/plasma_store',
 'raylet_socket_name': '/scratch/sergei/tmp/session_2020-07-17_19-50-37_777157_55814/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/scratch/sergei/tmp/session_2020-07-17_19-50-37_777157_55814'}

In [3]:
tf.keras.backend.set_floatx('float32')

In [30]:
env_cls = create_env
env_config = {'with_video': True}#True}

def build_trainer_config(restore_state=None, train_policies=None, config=None):
    """Build configuration for 1 run."""
    obs_space = env_cls(env_config).observation_space
    act_space = env_cls(env_config).action_space

    policy_template = "player_%d"

    def get_agent_config(agent_id):
        agent_config_pretrained = (PPOTFPolicy, obs_space, act_space, {
            'model': {
                        "custom_model": "GymCompetePretrainedModel",
                        "custom_model_config": {
                            "agent_id": agent_id - 1,
                            "env_name": env_name,
                            "model_config": {},
                            "name": "model_%s" % (agent_id - 1)
                        },           
                        
                    },
            
            "framework": "tfe",
        })
        
        agent_config_from_scratch = (PPOTFPolicy, obs_space, act_space, {
                    "model": {
                        "use_lstm": False,
                        "fcnet_hiddens": [64, 64],
                        #"custom_action_dist": "DiagGaussian",
                    },
                    "framework": "tfe",
                })
        
        if agent_id == 1:
            return agent_config_from_scratch
        elif agent_id == 2:
            return agent_config_pretrained
        else:
            raise KeyError("Wrong agent id %s" % agent_id)

    N_POLICIES = 2

    policies = {policy_template % i: get_agent_config(i) for i in range(1, 1  + N_POLICIES)}
    policies_keys = list(sorted(policies.keys()))

    def select_policy(agent_id):
        assert agent_id in ["player_1", "player_2"]
        agent_ids = ["player_1", "player_2"]
        
        # selecting the corresponding policy (only for 2 policies)
        return policies_keys[agent_ids.index(agent_id)]

        # randomly choosing an opponent
        # return np.random.choice(list(policies.keys()))
    
    if train_policies is None:
        train_policies = list(policies.keys())
        
    for k in train_policies:
        assert k in policies.keys()

    config = {
        "env": env_name_rllib,
        "env_config": env_config,
    #    "gamma": 0.9,
      "num_workers": 0,
    #  "num_envs_per_worker": 10,
    #   "rollout_fragment_length": 10,
       "train_batch_size": config['train_batch_size'],
        "multiagent": {
            "policies_to_train": train_policies,
            "policies": policies,
            "policy_mapping_fn": select_policy,
        },
        "framework": "tfe",
        #"train_batch_size": 512
        #"num_cpus_per_worker": 2
    }
    return config


def build_trainer(restore_state=None, train_policies=None, config=None):
    """Create a RPS trainer for 2 agents, restore state, and only train specific policies."""
    
    print("Using config")
    print(config)
    cls = PPOTrainer
    trainer = cls(config=config)
    env = trainer.workers.local_worker().env
    if restore_state is not None:
        trainer.restore_from_object(restore_state)
    return trainer

stats = []

def train(trainer, stop_iters, do_track=True):
    """Train the agents and return the state of the trainer."""
    for _ in range(stop_iters):
        results = trainer.train()
        print(pretty_print(results))
        if do_track:
            track.log(**results)
        global stats
        stats.append(results)
    o = trainer.save_to_object()
    return o

    


# try changing learning rate
config = {'train_batch_size': 256}

config['train_steps'] = 10

# ['humanoid_blocker', 'humanoid'],
config['train_policies'] = []#'player_1']
#config['num_workers'] = 32


print(config)
restore_state=None
rl_config = build_trainer_config(restore_state=restore_state,
                          train_policies=config['train_policies'],
                          config=config)
trainer = build_trainer(restore_state=None, config=rl_config)


{'train_batch_size': 256, 'train_steps': 10, 'train_policies': []}
Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents
Creating agent humanoid_blocker
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /scratch/sergei/better-adversarial-defenses/multiagent-competition/gym_compet

2020-07-17 21:41:33,704	WARNING util.py:37 -- Install gputil for GPU system monitoring.


In [21]:
import pickle, json, codecs
ck = json.load(open('/home/sergei/ray_results/./adversarial/train_one_3_2020-07-17_21-34-39m9qon2mn/checkpoint_0/checkpoint', 'r'))
w = codecs.decode(ck['weights'].encode(), 'base64')
trainer.set_weights(pickle.loads(w))

In [32]:
train(trainer, config['train_steps'], do_track=False)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.
custom_metrics: {}
date: 2020-07-17_21-43-20
done: false
episode_len_mean: 152.0
episode_reward_max: -0.3383652470135523
episode_reward_mean: -0.48690533688773474
episode_reward_min: -0.6354454267619172
episodes_this_iter: 2
episodes_total: 2
experiment_id: c6ee96fcb50744afb96244184244c6bf
hostname: svm
info:
  learner:
    player_1:
      cur_kl_coeff: 0.20000000298023224
      cur_lr: 4.999999873689376e-05
      en

FileNotFoundError: [Errno 2] No such file or directory: '/home/sergei/ray_results/PPO_YouShallNotPassHumans-v0_rllib_2020-07-17_21-41-32btsd_9s9/tmpoccvyew6save_to_object'

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist(np.array([y for x in stats for y in x['hist_stats']['policy_player_1_reward']]).flatten())

In [33]:
[x.close() for x in created_envs]

[None, None, None, None, None, None, None, None, None]

In [ ]:
trainer.evaluate()

In [ ]:
return

In [ ]:
from load_gym_compete_policy import get_policy_value_nets

In [ ]:
obs = env_cls(env_config).reset()

In [ ]:
acts = [trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)]

In [ ]:
np.mean(acts, axis=0)

In [ ]:
nets = get_policy_value_nets(env_name, 0)
policy_net_orig = nets['policy_mean_logstd_flat']

In [ ]:
m = trainer.get_policy('player_1').model
m.policy_net(obs['player_1'].reshape(1, -1))

In [ ]:
nets['policy_mean_logstd'](obs['player_1'].reshape(1, -1))

In [ ]:
mean, logstd = tf.split(policy_net_orig(obs['player_1'].reshape(1, -1)), 2, axis=1)

In [ ]:
np.mean([np.clip(np.random.normal(loc=mean[0], scale=np.exp(logstd[0]), size=(17,)), -0.4, 0.4) for _ in range(1000)], axis=0)

In [ ]:
np.mean([trainer.compute_action(obs['player_1'], policy_id='player_1') for _ in range(1000)], axis=0)

In [ ]:
trainer.compute_action(obs['player_1'], policy_id='player_1')

In [ ]:
env_cls(env_config).action_space.high

In [ ]:
trainer.config

In [16]:
import codecs

codecs.encode(pickle.dumps(np.array([1])), 'base64').decode()

'gANjbnVtcHkuY29yZS5tdWx0aWFycmF5Cl9yZWNvbnN0cnVjdApxAGNudW1weQpuZGFycmF5CnEB\nSwCFcQJDAWJxA4dxBFJxBShLAUsBhXEGY251bXB5CmR0eXBlCnEHWAIAAABpOHEIiYiHcQlScQoo\nSwNYAQAAADxxC05OTkr/////Sv////9LAHRxDGKJQwgBAAAAAAAAAHENdHEOYi4=\n'